In [2]:
import os

files = os.listdir("./images")


In [3]:
import cv2

location = "./images/"+files[0]


img = cv2.imread('./images/IMG_9109.JPG')
imgStar = cv2.imread('./images/star.png')
newH, newW = img.shape[:2]

BLACK = [0,0,0]

newStar = cv2.copyMakeBorder(imgStar,250,250,250,250,cv2.BORDER_CONSTANT, value= BLACK)

cv2.imwrite('newStar.jpg', newStar)




True

In [4]:
import math
class manipulate:
    def compareSize(img1,img2): 
        height1, width1, = img.shape[:2] #(height, width, num layers)
        height2, width2, = img2.shape[:2]
        # test case  Height 3456 * width 5184 = img 1
        # test case 512 * 512  = img 2
        difHeight = abs(height1 - height2)
        difWidth = abs(width1 - width2)
        return (difHeight,difWidth)
    
    def createImageToMatch(img,differance):
        #in test case 2944 , 4672
        # left width
        top, bottom = math.floor((differance[0])/2),math.ceil((differance[0])/2)
        right, left = math.floor((differance[1])/2),math.ceil((differance[1])/2)
        resizedImage = cv2.copyMakeBorder(img,top,bottom,left,right,cv2.BORDER_CONSTANT, value= [0,0,0])        
        return resizedImage
    def addImageToImage(img1,img2):
        dif = manipulate.compareSize(img1,img2)
        difHeight = (dif[0])//2
        difWidth = (dif[1])//2
        height, width = img2.shape[:2]
        for y in range(height):
            for x in range(width):
                if all(img2[y][x] != [0,0,0]):
                    img1[(y+difHeight)][(x+difWidth)] = img2[y][x]
        return img1
    

In [5]:
dif = manipulate.compareSize(img,imgStar)
newStar = manipulate.createImageToMatch(imgStar,dif)


newImage = manipulate.addImageToImage(img,newStar)
# cv2.imshow("check", newImage)
# cv2.imshow(-1)
cv2.imwrite("test.jpg",newImage)


True